### Базы данных: введение
### PostgreSQL HW

В следующих заданиях мы будем работать с базой данных, в которой имеется три таблицы:

1. Таблица с данными о пользователях (`user`):

`id` - уникальный идентификатор пользователя (primary key)

`gender`- пол

`age` - возраст

`country` - страна

`city` - город

`exp_group` - экспериментальная группа

`os` - операционная система

`source` - источник трафика

2. Таблица с данными о постах (`post`):

`id` - уникальный идентификатор поста (primary key)

`text` - текст поста

`topic` - тема поста

3. Таблица с данными о действиях пользователей (`feed_action`):

`user_id` (——>) `user` (`id`)- идентификатор пользователя     

`post_id` (——>) `post` (`id`)- идентификатор поста     

`action` - совершенное в сети действие     

`time` - время действия

In [1]:
!pip install psycopg2

In [2]:
import pandas as pd

In [3]:
conn_uri = '╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ'

### SELECT (1/2)
Сделайте запрос на все колонки из таблицы `user`. Для этого может понадобиться взять user в кавычки, вот так: `"user"` - чтобы PostgreSQL не спутал ее со своей встроенной таблицей user, где он ведет учет всех пользователей.

Напишите через пробел, с какого id начинаются записи и какой город у самого первого ID (на английском).

In [4]:
query1 =\
'''
SELECT *
FROM "user"
LIMIT 10
'''
df1 = pd.read_sql(query1, conn_uri)
df1

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
5,205,0,32,Russia,Dugulubgey,3,Android,ads
6,206,1,20,Russia,Kamenka,1,Android,ads
7,207,1,27,Russia,Groznyy,3,iOS,ads
8,208,1,17,Russia,Zima,1,Android,ads
9,209,0,17,Russia,Yuzhnouralsk,1,Android,ads


In [5]:
print(df1.iloc[0]['id'], df1.iloc[0]['city'])

200 Degtyarsk


### SELECT (2/2)
Отберите уникальные значения тем (`topic`) в таблице постов (`post`).
Напишите через пробел, сколько их и какая из них имеет самое длинное название.

In [6]:
query2 =\
'''
SELECT DISTINCT topic
FROM post
'''
df2 = pd.read_sql(query2, conn_uri)
df2

,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics
5,sport
6,business


### Фильтры (1/2)
Отберите пользователей старше 30 с устройством на iOS. Каков `ID` и `город` записи с минимальным ID (он идет первым в таблице)? Запишите значения через пробел.

In [7]:
query3 =\
'''
SELECT id, city
FROM "user"
WHERE age > 30 AND os = 'iOS'
ORDER BY id
'''
df3 = pd.read_sql(query3, conn_uri)
df3

,id,city
0,212,Podolsk
1,222,Ulyanovsk
2,224,Yekaterinburg
3,226,Moscow
4,243,Ekibastuz
...,...,...
17665,168517,Urzhum
17666,168525,Pavlohrad
17667,168536,Michurinsk
17668,168540,Tukan


In [8]:
print(df3.iloc[0]['id'], df3.iloc[0]['city'])

212 Podolsk


### Фильтры (2/2)
Отберите пользователей, которые не из России и при этом либо их экспериментальная группа не лежит в (0, 3) или их город – Минск.

Напишите через пробел первые 4 ID в такой выдаче.

In [9]:
query4 =\
'''
SELECT id, country
FROM "user"
WHERE (country != 'Russia') AND (exp_group NOT IN (0, 3) OR city = 'Minsk')
'''
df4 = pd.read_sql(query4, conn_uri)
df4

,id,country
0,211,Belarus
1,213,Ukraine
2,216,Ukraine
3,218,Belarus
4,228,Azerbaijan
...,...,...
12471,168477,Turkey
12472,168524,Turkey
12473,168525,Ukraine
12474,168527,Ukraine


In [10]:
df4['id'].iloc[0:4]

0    211
1    213
2    216
3    218
Name: id, dtype: int64

### Группировка (1/6)
Подсчитайте средний возраст пользователей в разрезе страны. В ответ напишите средний возраст пользователей из Кипра. Ответ округлите до двух знаков после точки.

In [11]:
query5 =\
'''
SELECT country, ROUND(AVG(age), 2) AS mean_age
FROM "user"
GROUP BY country
ORDER BY mean_age, country
'''
df5 = pd.read_sql(query5, conn_uri)
df5

,country,mean_age
0,Azerbaijan,26.78
1,Finland,27.13
2,Russia,27.19
3,Kazakhstan,27.22
4,Latvia,27.22
5,Ukraine,27.26
6,Belarus,27.28
7,Turkey,27.45
8,Switzerland,27.95
9,Cyprus,28.07


In [12]:
df5.query('country == "Cyprus"').mean_age

9    28.07
Name: mean_age, dtype: float64

### Группировка (2/6)
Сгруппируйте пользователей сначала по экспериментальной группе, а затем по операционной системе. В каждой группе подсчитайте количество пользователей, максимальный и минимальный возраст.

Сохраните результаты в файл CSV колонками exp_group, os, total_users, max_age, min_age (обратите внимание на название последних трех колонок!), используя запятую в качестве сепаратора, и отправьте файл в LMS.

In [13]:
query6 =\
'''
SELECT exp_group, os, count(*) AS total_users, MAX(age) AS max_age, MIN(age) AS min_age
FROM "user"
GROUP BY exp_group, os

'''
df6 = pd.read_sql(query6, conn_uri)
df6

,exp_group,os,total_users,max_age,min_age
0,0,Android,21234,95,14
1,0,iOS,11489,79,14
2,1,Android,21232,92,14
3,1,iOS,11406,87,14
4,2,Android,21102,78,14
5,2,iOS,11512,74,14
6,3,Android,21319,84,14
7,3,iOS,11449,85,14
8,4,Android,21085,84,14
9,4,iOS,11377,78,14


In [14]:
df6.to_csv('output.csv', sep=',', index=False)

### Группировка (3/6)
В каждой категории постов определите длину самого длинного текста, используя таблицу `post`. В ответ отправьте тему, у которой длина самого большого поста равна по порядку 25 000.

In [15]:
query7 =\
'''
SELECT topic, MAX(LENGTH(text)) AS max_len
FROM post
GROUP BY topic
HAVING MAX(LENGTH(text)) > 25000
'''
df7 = pd.read_sql(query7, conn_uri)
df7

,topic,max_len
0,politics,25392


### Группировка (4/6)
Найдите страну с наименьшим числом пользователей среди стран, у которых общее число пользователей больше 1000. Введите ее название на английском.

In [16]:
query8 =\
'''
SELECT country, count(*) AS total_users
FROM "user"
GROUP BY country
HAVING count(*) > 1000
ORDER BY total_users;
'''
df8 = pd.read_sql(query8, conn_uri)
df8

,country,total_users
0,Azerbaijan,1542
1,Finland,1599
2,Turkey,1606
3,Kazakhstan,3172
4,Belarus,3293
5,Ukraine,8273
6,Russia,143035


### Группировка (6/6)
Отберите для пользователей из Москвы экспериментальные группы, в которых средний возраст больше `27.2`. Напишите через пробел число пользователей в этих группах.

In [17]:
query9 =\
'''
SELECT exp_group, count(*) AS total_users, AVG(age) AS mean_age
FROM "user"
WHERE city = 'Moscow'
GROUP BY exp_group
HAVING AVG(age) > 27.2
'''
df9 = pd.read_sql(query9, conn_uri)
df9

,exp_group,total_users,mean_age
0,1,4414,27.325102
1,3,4378,27.314299


In [18]:
df9['total_users'].sum()

8792

### Сортировка (1/2)
Сейчас мы будем работать с другой таблицей под названием `post`. Найдите в таблице `post` 3 темы с наибольшим количеством постов. Напишите через пробел названия этих тем.

In [19]:
query10 =\
'''
SELECT topic, count(*)
FROM post
GROUP BY topic
ORDER BY 2 DESC
LIMIT 3
'''
df10 = pd.read_sql(query10, conn_uri)
df10

,topic,count
0,movie,3000
1,covid,1799
2,sport,510


### Сортировка (2/2)
Отберите пользователей из Воронежа (`Voronezh`) и отсортируйте их по возрасту в убывающем порядке. Для равных возрастов отсортируйте по возрастанию экспериментальной группы.

In [20]:
query11 =\
'''
SELECT *
FROM "user"
WHERE city = 'Voronezh'
ORDER BY age DESC, exp_group ASC
'''
df11 = pd.read_sql(query11, conn_uri)
df11

,id,gender,age,country,city,exp_group,os,source
0,89990,0,71,Russia,Voronezh,0,Android,ads
1,2441,1,71,Russia,Voronezh,3,Android,ads
2,14184,1,66,Russia,Voronezh,3,Android,ads
3,35093,0,65,Russia,Voronezh,0,Android,ads
4,77284,0,64,Russia,Voronezh,4,Android,ads
...,...,...,...,...,...,...,...,...
1369,151841,1,14,Russia,Voronezh,4,Android,organic
1370,116446,0,14,Russia,Voronezh,4,iOS,organic
1371,18278,0,14,Russia,Voronezh,4,Android,ads
1372,110611,1,14,Russia,Voronezh,4,Android,organic


### Join (1/2)
Отберите `post_id`, `time`, `age` и `os` пользователя для всех лайков пользователей из Омска. Отсортируйте результаты по убыванию времени действия и загрузите в LMS первые 100 значений в формате CSV (сепаратор "запятая", колонки с теми же названиями, что и в задании).

In [32]:
user_query =\
'''
SELECT *
FROM "user"
'''
user_df = pd.read_sql(user_query, conn_uri)
user_df.head()

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [22]:
post_query =\
'''
SELECT *
FROM post
'''
post_df = pd.read_sql(post_query, conn_uri)
post_df.head()

,id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [23]:
feed_action_query =\
'''
SELECT *
FROM feed_action
LIMIT 10
'''
feed_action_df = pd.read_sql(feed_action_query, conn_uri)
feed_action_df.head()

,user_id,post_id,action,time
0,113387,6716,view,2022-01-19 17:48:54
1,93486,6789,like,2022-01-19 17:48:54
2,131500,6792,view,2022-01-19 17:48:54
3,125502,6791,view,2022-01-19 17:48:54
4,138364,6572,like,2022-01-19 17:48:54


In [26]:
query11 =\
'''
SELECT f.post_id, f.time, u.age, u.os
FROM "user" u
INNER JOIN feed_action f ON u.id = f.user_id
WHERE u.city = 'Omsk' AND f.action = 'like'
ORDER BY f.time DESC
LIMIT 100
'''
df11 = pd.read_sql(query11, conn_uri)
df11

,post_id,time,age,os
0,7159,2022-01-26 23:59:54,17,Android
1,7184,2022-01-26 23:56:30,17,Android
2,7265,2022-01-26 23:56:03,17,Android
3,7256,2022-01-26 23:55:24,17,Android
4,7080,2022-01-26 23:55:12,17,Android
...,...,...,...,...
95,7256,2022-01-26 22:43:54,31,Android
96,7316,2022-01-26 22:42:54,31,Android
97,7319,2022-01-26 22:39:35,36,iOS
98,7268,2022-01-26 22:39:04,36,iOS


In [28]:
df11.to_csv('output_1.csv', sep=',', index=False)

### Join (2/2)
Найдите город, пользователи которого в возрасте 36 лет 2021-12-01 числа просмотрели запись на тему `covid` меньше всего раз относительно других городов. Введите его название на английском.

In [31]:
query12 =\
'''
SELECT city, count(*)
FROM "user" u
JOIN feed_action f ON u.id = f.user_id
JOIN post p ON f.post_id = p.id
WHERE (age = 36) AND (topic = 'covid') AND (time::DATE = '2021-12-01') AND (action = 'view')
GROUP BY city
ORDER BY count(*) ASC
LIMIT 10
'''
df12 = pd.read_sql(query12, conn_uri)
df12

,city,count
0,Arkhangelsk,5
1,Apsheronsk,8
2,Fryazino,8
3,Karmaskaly,9
4,Mariinsk,9
5,Varna,9
6,Kirs,10
7,Lubny,10
8,Kinel,11
9,Almetyevsk,11


### Топ постов
Напишите SQL запрос, используя который вы получите:
- топ N постов по лайкам,
- количество пролайкавших пользователей,
- время последнего лайка.

In [38]:
query13 =\
'''
SELECT post_id, count(*) as count_liked_users, MAX(time)
FROM "user" u
JOIN feed_action f ON u.id = f.user_id
WHERE action = 'like'
GROUP BY post_id
ORDER BY count_liked_users DESC
LIMIT 10
'''
df13 = pd.read_sql(query13, conn_uri)
df13

,post_id,count_liked_users,max
0,4866,31714,2022-01-01 13:57:43
1,5127,29637,2022-01-05 13:52:17
2,6448,27939,2022-01-22 05:50:39
3,6498,27729,2022-01-22 21:45:11
4,6513,27686,2022-01-22 19:59:52
5,3192,26507,2021-12-06 19:57:50
6,6314,26478,2022-01-20 07:46:21
7,6864,24156,2022-01-26 23:52:48
8,5413,23441,2022-01-07 09:57:07
9,5165,22596,2022-01-04 17:43:12
